<a href="https://colab.research.google.com/github/travelmail26/rasabotpublic/blob/main/whisper_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:


!pip install transformers
!pip install pydub
!pip install whisper
!pip install IPYthon
!pip install datasets
!pip install pytorch-lightning
!pip install IPYthon

!pip install -U openai-whisper
!pip install whisper

!pip install huggingface_hub
!pip install pytorch-lightning
!pip install gradio
!pip install datasets[audio]
!pip install evaluate
!pip install sentencepiece


In [2]:
from huggingface_hub import notebook_login
notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [3]:
import transformers
from transformers import WhisperConfig 
import pandas as pd
import numpy as np
import torchaudio
import torch
import flask
from pydub import AudioSegment
import matplotlib.pyplot as plt
import math
import whisper
from IPython.display import Audio
from torchaudio.utils import download_asset
from transformers import TFWhisperModel, Seq2SeqTrainingArguments, WhisperFeatureExtractor
from transformers import WhisperProcessor, WhisperForConditionalGeneration,WhisperTokenizer
from transformers import AutoFeatureExtractor, WhisperModel, DataCollatorWithPadding
from torchaudio import transforms
from datasets import Audio
import evaluate
from transformers import pipeline

from scipy.io import wavfile
import scipy.io.wavfile as wavfile
from datasets import load_dataset, DatasetDict, Audio, interleave_datasets, IterableDatasetDict


from pytorch_lightning import LightningModule
from pytorch_lightning import Trainer, seed_everything
from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# New Section

In [ ]:
input_transcribe = list("/content/Voice_to_Text/Input/transcribe.txt")
print(len(input_transcribe))

43


In [ ]:
#Now we need to create an input file with Audio and Transcribe to train with whisper model. 

In [ ]:
 input_audio_path_list = []
 input_audio_list = []
 mel_list = []
 waveform_list = []
 audio_size = 0
 print(whisper.__file__)
 wmodel = whisper.load_model("base")

/usr/local/lib/python3.9/dist-packages/whisper/__init__.py


100%|███████████████████████████████████████| 139M/139M [00:01<00:00, 91.1MiB/s]


In [7]:
#df = pd.DataFrame(
#{"audio": ["/content/Voice_to_Text/Input/1.wav", "/content/Voice_to_Text/Input/2.wav", ..., "/content/Voice_to_Text/Input/45.wav"]})

#audio_dataset = Dataset.from_pandas(df)

#audio_dataset = audio_dataset.cast_column("audio", Audio())

input=DatasetDict()

input['voice_train'] = load_dataset("audiofolder", data_dir="/content/Voice_to_Text", split='train',streaming=True)
print(input)


Resolving data files:   0%|          | 0/46 [00:00<?, ?it/s]

DatasetDict({
    voice_train: <datasets.iterable_dataset.IterableDataset object at 0x7f40dd967640>
})


In [10]:

input["common_train"] = load_dataset("mozilla-foundation/common_voice_11_0", "en", split="train", use_auth_token=True,streaming=True)
input["common_validation"] = load_dataset("mozilla-foundation/common_voice_11_0", "en", split="validation", use_auth_token=True,streaming=True)
input["common_test"] = load_dataset("mozilla-foundation/common_voice_11_0", "en", split="test", use_auth_token=True,streaming=True)


In [ ]:
print(common_voice['train'])
#print(next(iter(common_voice['train'])))#

In [11]:
for k,v in input.items():
        input[k] = v.cast_column("audio", Audio(sampling_rate=16000))
        input[k] = input[k].remove_columns(
            set(input[k].features.keys()) - set(["audio", "text"])
        )




In [12]:
print(input)

DatasetDict({
    voice_train: <datasets.iterable_dataset.IterableDataset object at 0x7f40dcfcdca0>
    common_train: <datasets.iterable_dataset.IterableDataset object at 0x7f40dcfb2f70>
    common_validation: <datasets.iterable_dataset.IterableDataset object at 0x7f40dd03cd00>
    common_test: <datasets.iterable_dataset.IterableDataset object at 0x7f40dd039f10>
})


In [13]:

data_dict = IterableDatasetDict()

data_dict['train'] = interleave_datasets([
                    input['common_train'],
                    input['voice_train'],
                    input['common_validation']], stopping_strategy='all_exhausted')


data_dict['test'] = input['common_test']

print(data_dict)

{'train': <datasets.iterable_dataset.IterableDataset object at 0x7f40dd9631f0>, 'test': <datasets.iterable_dataset.IterableDataset object at 0x7f40dd039f10>}


In [ ]:
Dataset_audio = input.cast_column("audio", Audio(sampling_rate=16000))
print(Dataset_audio['train'][0])

{'audio': {'path': '/content/Voice_to_Text/Input/1.wav', 'array': array([ 0.00276429,  0.00529364,  0.00454676, ..., -0.00354558,
       -0.00468975,  0.        ], dtype=float32), 'sampling_rate': 16000}, 'text': 'The Internet is a vast network that connects computers all over the world. Through the Internet, people can share information and communicate from anywhere around the world.'}


In [ ]:
print(next(iter(input['common_train'])))

In [ ]:
input_str = input['common_train'][0]['text']
print(input_str)
labels = tokenizer(input_str).input_ids
print(labels)
decoded_with_special = tokenizer.decode(labels, skip_special_tokens=False)
print(decoded_with_special)
decoded_str = tokenizer.decode(labels, skip_special_tokens=True)
print(decoded_str)
print(f"Are equal:             {input_str == decoded_str}")

In [51]:
kwargs={
    "text": "data_dict['text']",
    "text_target":"data_dict['text']"
}

In [ ]:
feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")

tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="English", task="transcribe")

processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="English", task="transcribe")

In [75]:
def prepare_dataset(batch):
    # load and resample audio data from 48 to 16kHz
    audio = batch["audio"]

    # encode target text to label ids 
    batch["labels"] = tokenizer(batch["text"], text_target=batch['text']).input_ids
    
    # compute log-Mel input features from input audio array 
    batch["input_features"] = processor.feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"],text=batch['text']).input_features[0]

    
    return batch


Dataset = data_dict.map(prepare_dataset)


In [33]:
print(Dataset)

{'train': <datasets.iterable_dataset.IterableDataset object at 0x7f40dcfcdcd0>, 'test': <datasets.iterable_dataset.IterableDataset object at 0x7f40dcfcdd30>}


In [76]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt", text=batch['text'])

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt", text=batch['text'])

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch
data_collator =  DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

In [77]:
def compute_metrics(pred):
    metric = evaluate.load("wer")
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [78]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper_small_en",  # change to a repo name of your choice
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-5,
    warmup_steps=500,
    max_steps=100,
    num_train_epochs=100,
    gradient_checkpointing=True,
    fp16=False,
    evaluation_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=1000,
    eval_steps=1000,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=True,
    remove_unused_columns=False,
    ignore_data_skip=True
)

PyTorch: setting up devices


In [79]:
#processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="English", task="transcribe", **kwargs)
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--openai--whisper-small/snapshots/358bfe95c5fa1a1bc953f29e7d36170006c40fbc/config.json
Model config WhisperConfig {
  "_name_or_path": "openai/whisper-small",
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "architectures": [
    "WhisperForConditionalGeneration"
  ],
  "attention_dropout": 0.0,
  "begin_suppress_tokens": [
    220,
    50257
  ],
  "bos_token_id": 50257,
  "d_model": 768,
  "decoder_attention_heads": 12,
  "decoder_ffn_dim": 3072,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 12,
  "decoder_start_token_id": 50258,
  "dropout": 0.0,
  "encoder_attention_heads": 12,
  "encoder_ffn_dim": 3072,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 12,
  "eos_token_id": 50257,
  "forced_decoder_ids": [
    [
      1,
      50259
    ],
    [
      2,
      50359
    ],
    [
      3,
      50363
    ]
  ],
  "init_std": 0.02,
  "is_encoder_decoder": true,
  "max_length": 448

In [80]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=Dataset['train'],
    eval_dataset=Dataset['train'],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    #tokenizer=tokenizer
    tokenizer=processor,

)

/content/./whisper_small_en is already a clone of https://huggingface.co/pnandhini/whisper_small_en. Make sure you pull the latest changes with `repo.git_pull()`.
max_steps is given, it will override any value given in num_train_epochs


In [81]:
#model.gradient_checkpointing_enable()
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1600
  Num Epochs = 9223372036854775807
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 4
  Total optimization steps = 100
  Number of trainable parameters = 241734912
Reading metadata...: 948736it [00:13, 69769.25it/s]


ValueError: ignored

In [ ]:
trainer.save_model() 

Saving model checkpoint to ./whisper_small_en
Configuration saved in ./whisper_small_en/config.json
Configuration saved in ./whisper_small_en/generation_config.json
Model weights saved in ./whisper_small_en/pytorch_model.bin
Feature extractor saved in ./whisper_small_en/preprocessor_config.json
Saving model checkpoint to ./whisper_small_en
Configuration saved in ./whisper_small_en/config.json
Configuration saved in ./whisper_small_en/generation_config.json
Model weights saved in ./whisper_small_en/pytorch_model.bin
Feature extractor saved in ./whisper_small_en/preprocessor_config.json
Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Automatic Speech Recognition', 'type': 'automatic-speech-recognition'}, 'dataset': {'name': 'audiofolder', 'type': 'audiofolder', 'config': 'default', 'split': 'train', 'args': 'default'}}
Several commits (2) will be pushed upstream.
The progress bars may be unreliable.
batch response: Authorization error.
error

In [ ]:
#loader = torch.utils.data.DataLoader(Dataset, batch_size=2, collate_fn=DataCollatorWhithPadding())
test_Dataset =  getData(test_data_list, tokenizer, sample_rate)
#loader= trainer.get_test_dataloader(Dataset)


In [ ]:
kwargs = {
    "dataset_tags": "voice audio sample",
    "dataset": "random talks",  # a 'pretty' name for the training dataset
    "dataset_args": "config: en, split: train",
    "language": "en",
    "model_name": "/pnandhini/whisper_small_en",  # a 'pretty' name for our model
    "finetuned_from": "openai/whisper-small",
    "tasks": "automatic-speech-recognition",
    "tags":  "hf-asr-leaderboard",
  
}

In [ ]:
#trainer.push_to_hub(**kwargs)
##trainer.push_to_hub("/pnandhini/whisper_small_en")
model.push_to_hub("pnandhini/whisper_small_en", create_pr=1)

Configuration saved in whisper_small_en/config.json
Configuration saved in whisper_small_en/generation_config.json
Model weights saved in whisper_small_en/pytorch_model.bin
Uploading the following files to pnandhini/whisper_small_en: generation_config.json,pytorch_model.bin,config.json


Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

pytorch_model.bin:   0%|          | 0.00/967M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/pnandhini/whisper_small_en/commit/ef7447db5694fb7ce5bacf913f7ec9a7d7640a14', commit_message='Upload WhisperForConditionalGeneration', commit_description='', oid='ef7447db5694fb7ce5bacf913f7ec9a7d7640a14', pr_url='https://huggingface.co/pnandhini/whisper_small_en/discussions/6', pr_revision='refs/pr/6', pr_num=6)

In [ ]:
tokenizer.push_to_hub("pnandhini/whisper_small_en", create_pr = 1)

tokenizer config file saved in whisper_small_en/tokenizer_config.json
Special tokens file saved in whisper_small_en/special_tokens_map.json
added tokens file saved in whisper_small_en/added_tokens.json
Uploading the following files to pnandhini/whisper_small_en: special_tokens_map.json,normalizer.json,added_tokens.json,vocab.json,tokenizer_config.json,merges.txt


CommitInfo(commit_url='https://huggingface.co/pnandhini/whisper_small_en/commit/304ba6ca54cc4070fc9946b42426e2528346e14b', commit_message='Upload tokenizer', commit_description='', oid='304ba6ca54cc4070fc9946b42426e2528346e14b', pr_url='https://huggingface.co/pnandhini/whisper_small_en/discussions/8', pr_revision='refs/pr/8', pr_num=8)

In [ ]:

dataset = load_dataset("audiofolder", data_files="/content/Voice_to_Text/Input/1.wav", split="train")
dataset = dataset.cast_column("audio", Audio(sampling_rate=16000))
sampling_rate = dataset.features["audio"].sampling_rate
audio_file = dataset[0]["audio"]["path"]

transcriber = pipeline("automatic-speech-recognition",model ="pnandhini/whisper_small_en")
transcriber(audio_file)

Extracting data files: 0it [00:00, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset audiofolder downloaded and prepared to /root/.cache/huggingface/datasets/audiofolder/default-5baf93baa3e50d89/0.0.0/6cbdd16f8688354c63b4e2a36e1585d05de285023ee6443ffd71c4182055c0fc. Subsequent calls will reuse this data.


/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1273: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 448 (`generation_config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


{'text': ' ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ ॐ'}

In [ ]:
transcriber = pipeline("automatic-speech-recognition",model ="openai/whisper_small", use_auth_token=True)
transcriber(audio_file)

OSError: ignored

In [ ]:
from transformers import pipeline
import gradio as gr

pipe = pipeline(model="pnandhini/whisper_small_en")  # change to "your-username/the-name-you-picked"

def transcribe(audio):
    text = pipe(audio)["text"]
    print(text)
    return text

example = [['/content/Voice_to_Text/Input/1.wav']]
iface = gr.Interface(
    fn=transcribe, 
    #gr.inputs.Audio(type="numpy", label="Input"),
    inputs=gr.Audio(type="numpy", label="Input"),
    outputs=gr.Audio(type="numpy",label="Vocals"),
    examples=example,
    title="Whisper Small English",
    description="Realtime demo for English speech recognition using a fine-tuned Whisper small model.",
)
  #  outputs="text",

iface.launch()

Streaming output truncated to the last 5000 lines.
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
   

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>